In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""A Flask app that uses Vertex AI and Nominatim to get address coordinates.

This app takes an address as input and uses the Vertex AI Gemini model to
extract relevant location information. It then uses the Nominatim API to
retrieve the coordinates for the address.
"""

import json
import logging
import os

from flask import Flask, render_template, request
import requests
import vertexai
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
    Part,
)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate("goblob-95e2a-0e1184d35308.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()

PROJECT_ID = "goblob-95e2a"

vertexai.init(
    project=PROJECT_ID,
    location="us-central1",
    staging_bucket="gs://aiuda-docs",
)

logger = logging.getLogger(__name__)

get_service_categories = FunctionDeclaration(
    name="get_service_categories",
    description="Get service categories from the database",
    parameters={
        "type": "object",
        "properties": {},
    },
)

get_service_provider = FunctionDeclaration(
    name="get_service_provider",
    description="Get service providers based on the tags",
    parameters={
        "type": "object",
        "properties": {
            "tag": {
                "type": "string",
                "description": "the category of the service the user is looking for",
            }
        },
    },
)

yomap_tool = Tool(
    function_declarations=[get_service_categories, get_service_provider],
)


def get_service_categories_from_firebase():
    tags_ref = (
        db.collection("tags")
        # .where(filter=FieldFilter("active", "==", True))
        # .where(filter=FieldFilter("rating", ">=", 3))
    )
    docs = tags_ref.stream()

    tags = []
    for doc in docs:
        if "text" in doc.to_dict().keys():
            tags.append(doc.to_dict()["text"])
    return tags


def get_service_provider_from_firebase(tag: str):
    profile = db.collection("profiles")
    print(tag)
    docs = profile.where("service.text", "==", tag["tag"]).get()
    return [doc.to_dict()["displayName"] for doc in docs]


function_handler = {
    "get_service_categories": get_service_categories_from_firebase,
    "get_service_provider": get_service_provider_from_firebase,
}

gemini_model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[yomap_tool],
)

In [2]:
from IPython.display import display, Markdown

In [3]:
chat = gemini_model.start_chat()

In [4]:
chat.send_message("Cuales son las categorias de la base de datos?")

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_service_categories"
        args {
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.067546688
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0682885423
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.121790156
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.199308
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.107437477
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0409990288
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0824015141
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0442658812
  }
}
usage_metadata {


In [5]:
chat.send_message("Puedes buscarme un plomero?")

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_service_provider"
        args {
          fields {
            key: "tag"
            value {
              string_value: "plumbing"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0782188699
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0632505193
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.302391082
    severity: HARM_SEVERITY_LOW
    severity_score: 0.336767137
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.150777549
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0484059565
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    proba

In [4]:
def send_chat_message(prompt):
    display(Markdown("#### Prompt"))
    print(prompt, "\n")
    prompt += """
    Give a concise, high-level summary. Only use information that you learn from 
    the API responses. Always search and response in the same language the user 
    is asking.
    """

    categories = get_service_categories_from_firebase()

    prompt += (
        """ Cuando el usuario pregunte por un servicio verifica primero si la categoria de servicio
    solicitada esta en esta lista: """
        + str(categories)
        + ". En caso de que no este busca la categoria mas parecida"
    )

    # Send a chat message to the Gemini API
    response = chat.send_message(prompt)

    # Handle cases with multiple chained function calls
    function_calling_in_process = True
    while function_calling_in_process:
        # Extract the function call response
        function_call = response.candidates[0].content.parts[0].function_call

        # Check for a function call or a natural language response
        if function_call.name in function_handler.keys():
            # Extract the function call
            function_call = response.candidates[0].content.parts[0].function_call

            # Extract the function call name
            function_name = function_call.name
            display(Markdown("#### Predicted function name"))
            print(function_name, "\n")

            if function_name == "get_service_categories":
                # Invoke a function that calls an external API
                function_api_response = function_handler[function_name]()
                display(Markdown("#### API response"))
                print(function_api_response[:500], "...", "\n")
            else:
                # Extract the function call parameters
                params = {key: value for key, value in function_call.args.items()}
                display(Markdown("#### Predicted function parameters"))
                print(params, "\n")

                # Invoke a function that calls an external API
                function_api_response = function_handler[function_name](params)[
                    :20000
                ]  # Stay within the input token limit
                display(Markdown("#### API response"))
                print(function_api_response[:500], "...", "\n")

            # Send the API response back to Gemini, which will generate a natural language summary or another function call
            response = chat.send_message(
                Part.from_function_response(
                    name=function_name,
                    response={"content": function_api_response},
                ),
            )
        else:
            function_calling_in_process = False

    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\$")))

In [7]:
send_chat_message("Puedes decirme cuales son las categorias?")

#### Prompt

Puedes decirme cuales son las categorias? 



#### Predicted function name

get_service_categories 



#### API response

['misc', 'cerrajeria', 'piano', 'cabello', 'bateria', 'french_food', 'body_treatments', '8_56_hierro', 'turistico', 'home', 'moto', 'filmmaking', 'auto', 'music', 'video', 'aseo_completo_de_casas', 'cubana', 'uber', 'tutorias', 'muebles', 'gatos', 'transporte_de_alimentos', 'smartphone', 'carrozas', 'plomero', 'taxi', 'delivery', 'equipos_de_sonido', 'hamburguesas', '5', 'education', 'tv', 'motoboy', 'piano', 'musica', 'albanil', 'depilacion', 'llaves_perdidas', 'mototaxi', 'guitar', 'restaurant', 'moda', 'food', 'manos_y_pies', 'indriver', 'cerrajeria_automotriz', 'foodtruck', 'marketing', 'electeicidade_generale', 'repair', 'jardin', 'travel_agency', 'transport', 'piano', 'alba', 'musica', 'computadoras', 'pies', 'bateria', 'nevera', 'musico', 'speak_english', 'doctor', 'piano', 'psicopedagoga', 'electricidade', 'bicicletas', 'aire_aconditionado', 'estufas', 'musica', 'educacion', 'planchas', 'dog_sitting', '8_56_hierro', 'desarrollo_de_software', 'taxi', 'clases_de_ingles', 'speak_e

#### Natural language response

Las categorias son: misc, cerrajeria, piano, cabello, bateria, french_food, body_treatments, 8_56_hierro, turistico, home, moto, filmmaking, auto, music, video, aseo_completo_de_casas, cubana, uber, tutorias, muebles, gatos, transporte_de_alimentos, smartphone, carrozas, plomero, taxi, delivery, equipos_de_sonido, hamburguesas, 5, education, tv, motoboy, piano, musica, albanil, depilacion, llaves_perdidas, mototaxi, guitar, restaurant, moda, food, manos_y_pies, indriver, cerrajeria_automotriz, foodtruck, marketing, electericidade_generale, repair, jardin, travel_agency, transport, piano, alba, musica, computadoras, pies, bateria, nevera, musico, speak_english, doctor, piano, psicopedagoga, electricidade, bicicletas, aire_acondicionado, estufas, musica, educacion, planchas, dog_sitting, 8_56_hierro, desarrollo_de_software, taxi, clases_de_ingles, speak_english, lavado_de_automoviles, clases_de_musica, portuguese, speak_english, pizza, aseo, maquina_de_coser, english, ar_conditionado_ac, bateria, delivery, nachos, privado, spa, linea_blanca, paseo_de_aves, aire_acondicionado, vegetariano, rapidass, testelectricista, pet_care, test, mascotas, health_panama, medico_a_domicilio, user, iphone, musica, electricista, piano, pets, fisioterapia, nueva_salud, photography, costurera, plantas, appliance, bateria, test, plomero, musica, privado, 1, android, albanil, health, ballo, street_food, masajes, paseo_de_perros, piano, llaves_para_autos, entrega. 


In [5]:
send_chat_message("Cuales son los proveedores de servicio spa?")

#### Prompt

Cuales son los proveedores de servicio spa? 



#### Predicted function name

get_service_provider 



#### Predicted function parameters

...

/home/nmlemus/projects/ai-llm/.conda/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


#### API response

['Carolina', 'Angela ', 'Emulator 66980917'] ... 



#### Natural language response

Los proveedores de servicios de spa son Carolina, Angela y Emulator 66980917. 


In [6]:
send_chat_message("Alguien con servicio span?")

#### Prompt

Alguien con servicio span? 



#### Predicted function name

get_service_provider 



#### Predicted function parameters

{'tag': 'spa'} 

{'tag': 'spa'}


#### API response

['Carolina', 'Angela ', 'Emulator 66980917'] ... 



#### Natural language response

Sí, Carolina, Angela y Emulator 66980917 ofrecen servicios de spa. 
